In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [70]:
new_url = "https://coinmarketcap.com/"
new_result = requests.get(new_url).text
new_doc = BeautifulSoup(new_result, "html.parser")

tbody = new_doc.tbody
trs = tbody.contents
BTC_tr = trs[0]
ETH_tr = BTC_tr.next_sibling
coins = {}

for tr in trs[:10]:
    name, price, day_change, week_change, market_cap, volume, circulating_supply= tr.contents[2:9]
    name = name.p.string
    price = price.a.string
    day_change, week_change = day_change.text, week_change.text
    market_cap = "$" + market_cap.text.split("$")[1]
    volume = volume.findAll("p")[1].text
    circulating_supply = circulating_supply.text
    coins[name] = [price, day_change, week_change, market_cap, volume, circulating_supply]
    
df = pd.DataFrame.from_dict(coins, orient= "index", columns = ["price", "day_change", "week_change", "market_cap", "volume", "circulating_supply"])
df

,price,day_change,week_change,market_cap,volume,circulating_supply
Bitcoin,"$45,947.97",0.78%,8.55%,$864.45B,"611,827 BTC","18,813,762 BTC"
Ethereum,"$3,419.67",3.02%,12.59%,$401.79B,"5,092,256 ETH","117,494,266 ETH"
Cardano,$2.67,1.73%,6.86%,$85.52B,"2,721,125,383 ADA","32,038,100,544 ADA"
Binance Coin,$418.67,1.46%,15.49%,$70.39B,"3,762,457 BNB","168,137,036 BNB"
Tether,$0.9999,0.03%,0.04%,$68.34B,"71,378,492,965 USDT","68,340,615,736 USDT"
Solana,$180.12,4.55%,31.99%,$52.84B,"18,222,161 SOL","293,374,666 SOL"
XRP,$1.11,0.79%,14.74%,$51.58B,"3,057,137,269 XRP","46,585,282,244 XRP"
Polkadot,$34.46,12.73%,3.69%,$34.04B,"95,571,321 DOT","987,579,315 DOT"
Dogecoin,$0.2458,0.19%,19.40%,$32.26B,"4,607,961,167 DOGE","131,255,374,699 DOGE"
USD Coin,$0.9999,0.03%,0.01%,$29.24B,"2,288,603,387 USDC","29,245,020,794 USDC"


In [33]:
#gpu = input("What product do you want to search for?")
search_term = "3070"
url = f"https://www.newegg.com/p/pl?d={search_term}&N=4131"
page = requests.get(url).text
doc = BeautifulSoup(page, "html.parser")

page_text = doc.find(class_="list-tool-pagination-text").strong
pages = int(str(page_text).split("/")[-2].split(">")[-1][:-1])

items_found = {}

for page in range(1, pages + 1):
    url = f"https://www.newegg.ca/p/pl?d={search_term}&N=4131&page={page}"
    page = requests.get(url).text
    doc = BeautifulSoup(page, "html.parser")

    div = doc.find(class_="item-cells-wrap border-cells items-grid-view four-cells expulsion-one-cell")
    items = div.find_all(text=re.compile(search_term))

    for item in items:
        parent = item.parent
        if parent.name != "a":
            continue

        link = parent['href']
        next_parent = item.find_parent(class_="item-container")
        try:
            price = next_parent.find(class_="price-current").find("strong").string
            items_found[item] = {"price": int(price.replace(",", "")), "link": link}
        except:
            pass

sorted_items = sorted(items_found.items(), key=lambda x: x[1]['price'])

for item in sorted_items:
    print(item[0])
    print(f"${item[1]['price']}")
    print(item[1]['link'])
    print("-------------------------------")

EVGA GeForce RTX 3070 FTW3 ULTRA GAMING Video Card, 08G-P5-3767-KL, 8GB GDDR6, iCX3 Technology, ARGB LED, Metal Backplate, LHR
$1449
https://www.newegg.ca/evga-geforce-rtx-3070-08g-p5-3767-kl/p/N82E16814487544?Description=3070&cm_re=3070-_-9SIA92GF4M3607-_-Product
-------------------------------
GIGABYTE Eagle GeForce RTX 3070 Ti 8GB GDDR6X PCI Express 4.0 ATX Video Card GV-N307TEAGLE-8GD
$1529
https://www.newegg.ca/gigabyte-geforce-rtx-3070-ti-gv-n307teagle-8gd/p/N82E16814932445?Description=3070&cm_re=3070-_-9SIA92GFRH1860-_-Product
-------------------------------
Colorful iGame GeForce RTX 3070 Ti Advanced OC 8G GDDR6X PCI Express 4.0 x16 Video Card,RTX 3070 Ti Advanced OC 8G
$1574
https://www.newegg.ca/p/1FT-005B-000C0?Description=3070&cm_re=3070-_-1FT-005B-000C0-_-Product
-------------------------------
MSI Ventus GeForce RTX 3070 8GB GDDR6 PCI Express 4.0 Video Card RTX 3070 Ventus 2X 8G OC LHR
$1574
https://www.newegg.ca/p/1FT-0009-006M4?Description=3070&cm_re=3070-_-9SIB307FPZ21